In [1]:
import os
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
class FahrenheitTemperatures(Dataset):
    def __init__(self, start=-212, stop=212, size=5000):
        super(FahrenheitTemperatures, self).__init__()
        
        # Intialize local variables and scale the data
        self.scaler_f =  MinMaxScaler()
        self.scaler_c = MinMaxScaler()
        
        np.random.seed(42)
        f_temp = np.random.randint(start, high=stop, size=size).reshape(-1, 1)
        c_temp = np.array([self._f2c(f) for f in f_temp]).reshape(-1, 1)
        
        # transform and scale the X, y data 
        f_temp = self.scaler_f.fit_transform(f_temp)
        c_temp = self.scaler_c.fit_transform(c_temp)
        
        # convert to tensors
        self.X = torch.from_numpy(f_temp).float()
        self.y = torch.from_numpy(c_temp).float()
        self._samples = self.X.shape[0]
        
    def __getitem__(self, index):
        # support indexing such that dataset[i] can be used to get i-th sample
        # implement this python function for indexing
        return self.X[index], self.y[index] 
        
    def __len__(self):
        # we can call len(dataset) to return the size, so this can be used
        # as an iterator
        return self._samples
    
    def _f2c(self, f) -> float:
        return (f - 32) * 5.0/9.0
    
    @property
    def samples(self):
        return self._samples

In [4]:
class LinearNN(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_size=10):
        super(LinearNN, self).__init__()
        
        # Input, output, and hidden size paramaters
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Build the NN architecture
        self.l1 = torch.nn.Linear(input_size, output_size)
        
    def forward(self, x):
        out = self.l1(x)
        return out

In [5]:
# Let's now access our dataset
dataset = FahrenheitTemperatures()
first_dataset = dataset[0]
features, labels = first_dataset
samples = dataset.samples
print('Fahrenheit: {:.4f}'.format(features[0]))
print('Celcius   : {:.4f}'.format(labels[0]))
print('Samples   : {:.2f}'.format(samples))

Fahrenheit: 0.2411
Celcius   : 0.2411
Samples   : 5000.00


In [6]:
data_loader = DataLoader(dataset=dataset, batch_size=100, shuffle=True)

In [7]:
# our model, loss function, and optimizer
model = LinearNN(1, 1, 128)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
# Training loop
epochs= 1000
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(data_loader):
        
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs)
        
        # Compute and print loss
        loss = criterion(y_pred, labels)
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 100 == 0:
        print('Epoch: {}/{}, loss: {:.4f}, Weight: {:.4f}, bias: {:.4f}'
              .format(epoch + 1., epochs, loss.data.item(), model.l1.weight.item(), model.l1.bias.item()))

Epoch: 100.0/1000, loss: 0.0002, Weight: 1.0536, bias: -0.0287
Epoch: 200.0/1000, loss: 0.0001, Weight: 1.0275, bias: -0.0147
Epoch: 300.0/1000, loss: 0.0000, Weight: 1.0141, bias: -0.0076
Epoch: 400.0/1000, loss: 0.0000, Weight: 1.0072, bias: -0.0039
Epoch: 500.0/1000, loss: 0.0000, Weight: 1.0037, bias: -0.0020
Epoch: 600.0/1000, loss: 0.0000, Weight: 1.0019, bias: -0.0010
Epoch: 700.0/1000, loss: 0.0000, Weight: 1.0010, bias: -0.0005
Epoch: 800.0/1000, loss: 0.0000, Weight: 1.0005, bias: -0.0003
Epoch: 900.0/1000, loss: 0.0000, Weight: 1.0003, bias: -0.0002
Epoch: 1000.0/1000, loss: 0.0000, Weight: 1.0003, bias: -0.0001


In [9]:
with torch.no_grad():
    f_temp = np.arange(212, 185, -5).reshape(-1, 1)
    y_pred = model(torch.from_numpy(f_temp).float())
    print(y_pred)

tensor([[212.0564],
        [207.0551],
        [202.0537],
        [197.0524],
        [192.0511],
        [187.0497]])


In [ ]:
def f2c(f):
    return (f - 32) * 5.0/9.0

In [10]:
f_temp = np.arange(212, 185, -5).reshape(-1, 1)
f_temp

array([[212],
       [207],
       [202],
       [197],
       [192],
       [187]])

In [ ]:
f2c(187)

In [ ]:
f2c(-110)

In [ ]:
f2c(0)

In [11]:
from sklearn import preprocessing

import numpy as np
f_temp = f_temp = np.random.randint(-212, high=212, size=5000).reshape(-1, 1)
f_temp = preprocessing.normalize(f_temp, norm='l1')
f_temp

array([[ 1.],
       [-1.],
       [ 1.],
       ...,
       [-1.],
       [ 1.],
       [ 1.]])

In [17]:
f_temp = f_temp = np.random.randint(-212, high=212, size=5000).reshape(-1, 1)
f_temp.max(axis=0), f_temp.min(axis=0), f_temp.mean(axis=0),  f_temp.std(axis=0)

(array([211]), array([-212]), array([1.2]), array([122.80974228]))